In [116]:
import pypsa
import numpy as np
import pandas as pd
from datetime import datetime


"""
The network:

A central bus that connects a solar generator, an onshore and an offshore wind generator, a battery, and a load.

"""

# Create the timeseries index
index = pd.date_range("2011-01-01 00:00", "2011-12-31 21:00", freq="3h")

# Cost assumptions (example values, should be refined)
# TODO: get realistic figures here and in SEK or kSEK
cost_per_mw_solar = 1000  # in $/MW
cost_per_mw_wind = 1200  # in $/MW
cost_per_mwh_storage = 500  # in $/MWh

# Initialize the network
network = pypsa.Network()
network.set_snapshots(index)

# Add the buses
network.add("Bus", "Main bus")

# Add load
## Load the demand data (it comes in a format of peak demand per each hour and per month and per weekend/weekday and per h3 hexagon)
demand_data = pd.read_csv('data/demand/demand_vgr_2025.csv')
demand_year = demand_data['Year'][0]
demand_data.drop(columns=['Unnamed: 0', 'Year'], inplace=True)
demand_data['Timestamp'] = pd.to_datetime(demand_data['Timestamp'], format='%Y-%m-%d %H:%M:%S')
demand_data.set_index('Timestamp', inplace=True)

## Aggregate by summing over all the hexagons and resampling from 1h to 3h intervals
grouped_demand_data = demand_data.groupby([demand_data.index, 'Daytype']).sum()['Demand (MW)'].groupby([pd.Grouper(freq='3h', level=0), 'Daytype']).mean()

## Function that returns the appropriate demand for a given day
def select_demand(index):
    return grouped_demand_data[index.replace(year=demand_year ,day=1),'weekday' if index.weekday() < 5 else 'weekend']

## Build a load profile
target_percentage = 0.3 # This is the percentage of the demand that we want to fulfill
load_profile = [target_percentage*select_demand(i) for i in index]

## Finally add the load to the network
network.add("Load", "Desired load", bus="Main bus", p_set=load_profile)

# Add generation
# TODO: replace with data from atlite
solar_availability = np.random.rand(len(index))
wind_onshore_availability = np.random.rand(len(index))
wind_offshore_availability = np.random.rand(len(index))

network.add("Generator", "Solar park", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_solar, p_max_pu=solar_availability)

network.add("Generator", "Wind farm onshore", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_wind, p_max_pu=wind_onshore_availability)

network.add("Generator", "Wind farm offshore", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_wind, p_max_pu=wind_offshore_availability)

# Add storage
network.add("StorageUnit", "Battery", bus="Main bus", p_nom_extendable=True,
            capital_cost=cost_per_mwh_storage)

# Define the objective function (minimize cost)
network.optimize(network.snapshots, solver_name='cbc')

# Output results
print("Optimal size of Solar Park: {:.2f} MW".format(network.generators.p_nom_opt["Solar park"]))
print("Optimal size of Wind Farm (onshore): {:.2f} MW".format(network.generators.p_nom_opt["Wind farm onshore"]))
print("Optimal size of Wind Farm (offshore): {:.2f} MW".format(network.generators.p_nom_opt["Wind farm offshore"]))
print("Optimal size of Battery Storage: {:.2f} MWh".format(network.storage_units.p_nom_opt["Battery"]))


INFO:linopy.model: Solve problem using Cbc solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|█████████████████████████████████████████████████████| 6/6 [00:00<00:00, 155.68it/s]
INFO:linopy.io: Writing time: 0.33s
INFO:linopy.solvers:Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc -printingOptions all -import /tmp/linopy-problem-8j4q8370.lp -solve -solu /tmp/linopy-solve-mml3ktv0.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 23357 (-17527) rows, 17522 (-2) columns and 61311 (-17532) elements
Perturbing problem by 0.001% of 16555.536 - largest nonzero change 0.009316041 ( 0.00012137644%) - largest zero change 0.009315941
0  Obj 0 Primal inf 2485148 (2920)
341  Obj 183.92836 Primal inf 2608247 (3226)
682  Obj 348.53294 Primal inf 2758210 (3511)
1023  Obj 486.02441 Primal inf 2923432 (3804)
1364  Obj 627.03522 Primal inf 3015093.7 (4041)
1705  Obj 798.44027 Primal inf 298551

Optimal size of Solar Park: 1157.55 MW
Optimal size of Wind Farm (onshore): 502.90 MW
Optimal size of Wind Farm (offshore): 892.92 MW
Optimal size of Battery Storage: 1448.88 MWh
